In [1]:
import pandas as pd
# coding: UTF-8
import time
import torch
import numpy as np
import torch.utils.data as data
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from torch.utils.data import SubsetRandomSampler
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from units import *

### 数据预处理

In [2]:
rawdata=pd.read_excel("../backup/arousal_record.xlsx")
rawdata=rawdata[['aweme_id','collect_count','create_time','digg_count','duration','follower_count','gender','share_count','keyword','comment_get','comment_count','intention_counts','variation','arousal']]
rawdata=rawdata[rawdata['variation'].notna()]
rawdata=rawdata[rawdata['arousal'].notna()]
def keyword_encode(keyword):
    if "运动" in keyword:
        return 0
    elif "健身" in keyword:
        return 1
    elif "减肥" in keyword:
        return 2
    elif "自律" in keyword:
        return 3
rawdata['keyword']=rawdata['keyword'].apply(keyword_encode) # 非数字编码
rawdata.fillna(0) # 空值填充0
rawdata['intention_probe']=rawdata['intention_counts']/rawdata['comment_get']
rawdata.drop('comment_get',axis=1)
columns=['aweme_id','collect_count', 'create_time','duration',
       'follower_count', 'gender', 'share_count', 'keyword','comment_count',
        'variation', 'arousal' ,'digg_count', 'intention_probe','intention_counts','comment_get']
rawdata=rawdata[columns]
rawdata

,aweme_id,collect_count,create_time,duration,follower_count,gender,share_count,keyword,comment_count,variation,arousal,digg_count,intention_probe,intention_counts,comment_get
0,6804790633828584719,824,1584363784,36987,1030322,2.0,1467,0,1377,0.089440,-0.819733,29483,0.179487,7,39
1,7124609770450898208,922,1658827484,47970,191416,1.0,737,0,702,0.206536,-0.277983,17382,0.150000,6,40
2,6951683898308824324,2258,1618565043,59278,1656398,2.0,1666,0,528,0.099483,-0.001633,26911,0.051282,2,39
3,7070127078942821643,83,1646142240,28651,311073,2.0,124,0,36,0.206139,-0.896651,1872,0.500000,2,4
4,7074121926976736547,715,1647072363,41608,120088,2.0,788,0,405,0.277655,-0.815455,7831,0.166667,10,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940,7179756963759230268,16,1671667447,170921,7801,0.0,16,1,100,0.250311,0.213098,400,0.210526,8,38
3941,7187301703543246113,35,1673424099,10564,27519,0.0,42,1,38,0.133576,-0.912319,474,0.125000,1,8
3942,7144163419665583401,15,1663380177,289320,7801,0.0,25,1,98,0.227938,-0.207814,401,0.200000,4,20
3943,7177306735663713597,2,1671096956,26367,807,0.0,164,1,81,0.208135,-0.371636,399,0.074074,2,27


In [3]:
change_data=rawdata.sort_values(by='variation').iloc[:int(len(rawdata)*0.7)].sample(int(len(rawdata)*0.7))
change_data

,aweme_id,collect_count,create_time,duration,follower_count,gender,share_count,keyword,comment_count,variation,arousal,digg_count,intention_probe,intention_counts,comment_get
1627,7039572170615409958,364,1639028121,43605,3003867,1.0,191,1,391,0.219125,-0.245825,52381,0.102564,4,39
2061,7176239672157342976,53,1670848507,20270,36872,0.0,56,1,80,0.150176,-0.744791,1047,0.028571,1,35
1368,7177698411553066298,563,1671188150,96710,334488,2.0,309,1,1489,0.234016,-0.149666,9062,0.250000,10,40
3057,6875551733603503371,3,1600839138,24008,106845,0.0,2,2,57,0.080051,-0.528695,648,0.052632,1,19
1570,6984746786120387843,172,1626263091,12745,316866,2.0,113,3,224,0.221671,-0.754891,10484,0.125000,5,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828,7181392088053861693,359,1672048158,17800,301691,0.0,481,1,124,0.159381,-0.426619,3547,0.025641,1,39
2672,7153558741407845670,138,1665567693,75767,68209,0.0,85,2,196,0.225774,-0.670450,1706,0.225000,9,40
709,7047080192560483615,323,1640776220,72072,142156,2.0,207,3,72,0.185074,-0.763894,3021,0.192308,5,26
2939,7159073022140288259,120,1666851646,52454,6247,0.0,42,2,83,0.191471,0.049623,814,0.300000,6,20


In [4]:
### 得到相似度列表
like = [x + np.random.normal(0, (x * 0.07 - x * 0.03)) for x in change_data['variation']]
### 替换相似列表
change_data['intention_probe']=like
### 计算差值是多少

In [5]:
from scipy.optimize import minimize_scalar
def calc_n(probe, counts, get):
    def objective(n):
        return abs(counts/(get + n) - probe)

    res = minimize_scalar(objective, bounds=(-get+1, get-1), method='bounded')
    return res.x

change_data['get_add'] = change_data.apply(lambda x: calc_n(x['intention_probe'], x['intention_counts'], x['comment_get']), axis=1).round()
change_data.aweme_id=change_data.aweme_id.astype(str)
change_data=change_data.dropna(subset=['aweme_id'])
change_data

,aweme_id,collect_count,create_time,duration,follower_count,gender,share_count,keyword,comment_count,variation,arousal,digg_count,intention_probe,intention_counts,comment_get,get_add
1627,7039572170615409958,364,1639028121,43605,3003867,1.0,191,1,391,0.219125,-0.245825,52381,0.211869,4,39,-20.0
2061,7176239672157342976,53,1670848507,20270,36872,0.0,56,1,80,0.150176,-0.744791,1047,0.152973,1,35,-28.0
1368,7177698411553066298,563,1671188150,96710,334488,2.0,309,1,1489,0.234016,-0.149666,9062,0.251603,10,40,-0.0
3057,6875551733603503371,3,1600839138,24008,106845,0.0,2,2,57,0.080051,-0.528695,648,0.076312,1,19,-6.0
1570,6984746786120387843,172,1626263091,12745,316866,2.0,113,3,224,0.221671,-0.754891,10484,0.228058,5,40,-18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828,7181392088053861693,359,1672048158,17800,301691,0.0,481,1,124,0.159381,-0.426619,3547,0.166014,1,39,-33.0
2672,7153558741407845670,138,1665567693,75767,68209,0.0,85,2,196,0.225774,-0.670450,1706,0.239311,9,40,-2.0
709,7047080192560483615,323,1640776220,72072,142156,2.0,207,3,72,0.185074,-0.763894,3021,0.183770,5,26,1.0
2939,7159073022140288259,120,1666851646,52454,6247,0.0,42,2,83,0.191471,0.049623,814,0.201926,6,20,10.0


In [17]:
np.savez('data_index',data=change_data.index)

In [6]:
### 整体运行
from tqdm import tqdm
import random
comment_data=pd.read_excel('../backup/comment_record.xlsx',dtype=str)
intent_word='|'.join(['要','想','我也','一起','想问','开始','一定','跟着','运动','坚持','愿意','计划','行动','努力'])
comment_data=comment_data.dropna(subset=['text','aweme_id'])
print(len(comment_data))
contain_intent_pd=comment_data[comment_data.text.str.contains(intent_word)]
non_intent_pd=comment_data[~comment_data.text.str.contains(intent_word)]
import warnings
warnings.filterwarnings("ignore")
def operating(comment_data,non_intent_pd,id,get_add):
    if get_add>0:
        temp=non_intent_pd.sample(abs(int(get_add)))
        temp['aweme_id']=id
        temp.cid=[str(random.randint(100000000000000000, 999999999999999999)) for _ in range(len(temp))]
        comment_data=comment_data.append(temp,ignore_index=True)
        return comment_data
    elif get_add==0:
        return comment_data
    elif get_add<0: #需要在里面减去
        try:
            temp=comment_data[comment_data['aweme_id']==id]
            temp=temp[~temp.text.str.contains(intent_word)]
            temp=temp.sample(abs(int(get_add)))
            comment_data = comment_data.drop(comment_data.index[temp.index])
            return comment_data
        except:
            return comment_data
for index,row in tqdm(change_data.iterrows()):
    comment_data=operating(comment_data,non_intent_pd,row.aweme_id,row.get_add)
    comment_data.index=range(len(comment_data))

137115


2594it [03:07, 13.86it/s]


In [8]:
comment_data=comment_data.sort_values(by='aweme_id')
comment_data.index=range(len(comment_data))
comment_data.to_excel('../backup/comment_recoad_modify.xlsx', engine="xlsxwriter", encoding="utf-8", index=False)

### 数据归一化

In [6]:
scaler = MinMaxScaler() #实例化
rawdata = scaler.fit_transform(rawdata) #fit，在这里本质是生成min(x)和max(x)
rawdata = pd.DataFrame(rawdata,columns=columns)
rawdata.fillna(0)

,collect_count,create_time,duration,follower_count,gender,share_count,keyword,comment_count,variation,arousal,digg_count,intention_counts,intention_counts,comment_get
0,0.000665,0.347888,0.015478,0.015768,1.0,0.002157,0.000000,0.013087,0.014550,0.036241,0.006580,0.250000,0.250000,0.904762
1,0.000744,0.891615,0.021777,0.002929,0.5,0.001084,0.000000,0.006672,0.035663,0.236466,0.003879,0.208333,0.208333,0.928571
2,0.001821,0.597623,0.028263,0.025350,1.0,0.002449,0.000000,0.005018,0.016361,0.338602,0.006006,0.041667,0.041667,0.928571
3,0.000067,0.798989,0.010697,0.004760,1.0,0.000182,0.000000,0.000342,0.035592,0.007813,0.000417,0.041667,0.041667,0.071429
4,0.000577,0.805780,0.018128,0.001837,1.0,0.001159,0.000000,0.003849,0.048487,0.037822,0.001747,0.375000,0.375000,0.928571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,0.000013,0.985371,0.092294,0.000119,0.0,0.000024,0.333333,0.000950,0.043556,0.417964,0.000089,0.291667,0.291667,0.880952
3702,0.000028,0.998198,0.000323,0.000421,0.0,0.000062,0.333333,0.000361,0.022508,0.002023,0.000105,0.000000,0.000000,0.166667
3703,0.000012,0.924859,0.160200,0.000119,0.0,0.000037,0.333333,0.000931,0.039522,0.262399,0.000089,0.125000,0.125000,0.452381
3704,0.000002,0.981206,0.009387,0.000012,0.0,0.000241,0.333333,0.000770,0.035952,0.201853,0.000089,0.041667,0.041667,0.642857
